In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

uri = "mongodb+srv://<db_user>:<dbpwd>@cluster0.efy0v9p.mongodb.net/?appName=Cluster0"

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

import pymongo

db = client.sample_mflix


Pinged your deployment. You successfully connected to MongoDB!


Question 1

Question: From the movies collection, return the documents with the plot that starts with "war" in acending order of released date, print only title, plot and released fields. Limit the result to 5.

Answer:

In [50]:
movies = db.movies
for m in movies.find( { 'plot': { '$regex': 'war' } } ).sort('released', pymongo.ASCENDING).limit(5):
    print(f"{m['title']}, Plot: {m['plot']} was released in {m['released']}")


KeyError: 'released'

Question 2

Question: Group by rated and count the number of movies in each.

Answer:

In [51]:
stage_group_rated  = {
   "$group": {
         "_id": "$rated",
         "movie_count": { "$sum": 1 }, 
   }
}

pipeline = [
   stage_group_rated ,
]
results = movies.aggregate(pipeline)

for rated_summary in results:
   print(rated_summary)

{'_id': 'G', 'movie_count': 477}
{'_id': 'TV-G', 'movie_count': 59}
{'_id': 'PG', 'movie_count': 1852}
{'_id': None, 'movie_count': 9895}
{'_id': 'R', 'movie_count': 5537}
{'_id': 'APPROVED', 'movie_count': 709}
{'_id': 'TV-MA', 'movie_count': 60}
{'_id': 'Approved', 'movie_count': 5}
{'_id': 'GP', 'movie_count': 44}
{'_id': 'TV-PG', 'movie_count': 76}
{'_id': 'OPEN', 'movie_count': 1}
{'_id': 'TV-14', 'movie_count': 89}
{'_id': 'M', 'movie_count': 37}
{'_id': 'TV-Y7', 'movie_count': 3}
{'_id': 'AO', 'movie_count': 3}
{'_id': 'PASSED', 'movie_count': 181}
{'_id': 'Not Rated', 'movie_count': 1}
{'_id': 'PG-13', 'movie_count': 2321}


Question 3
Question: Count the number of movies with 3 comments or more.

Answer:

In [66]:
stage_lookup_comments = {
   "$lookup": {
         "from": "comments", 
         "localField": "_id", 
         "foreignField": "movie_id", 
         "as": "related_comments",
   }
}

stage_add_comment_count = {
   "$addFields": {
         "comment_count": {
            "$size": "$related_comments"
         }
   } 
}

# Match movie documents with at least 3 comment:
stage_match_with_comments = {
   "$match": {
         "comment_count": {
            "$gte": 3
         }
   } 
}

pipeline = [
   stage_lookup_comments,
   stage_add_comment_count,
   stage_match_with_comments,
]

results = movies.aggregate(pipeline)
for movie in results:
   print(movie["title"])
   print("Comment count:", movie["comment_count"])

   for comment in movie["related_comments"][:5]:
         print(" * {name}: {text}".format(
            name=comment["name"],
            text=comment["text"]))
   print()

KeyboardInterrupt: 